# Projet Recherche Opérationnelle

Antoine GICQUEL \
Victor BERTRET \
4ème année Génie Mathématique \
Annéé universitaire 2020 - 2021

*Importation des packages nécessaires:*

In [1]:
from pulp import* #Package pulp : résolution de modèles PLNE

## Introduction

Ce projet a été réalisé dans le cadre du module de Recherche Opérationnelle de notre formation d'ingénieur au département Génie Mathématique de l'INSA de Rennes. L'objectif de ce projet était d'étudier le fonctionnement algorithme de Branch-and-Bound appliqué au problème d'ordonnancement qui est le suivant. On considère un atelier de production devant usiner un ensemble $J = {1,...,n}$ de pièces. On suppose que l'atelier ne dispose que d'une machine (le problème peut tout à fait être étendu au cas où l'atelier dispose de plusieurs machines). Chaque pièce $j\in J$ nécessite $p_j$ unités de temps pour être usinée. De plus, à la date $t=0$, chaque pièce $j\in J$ doit être usinée avant la date $d_j$. Si ce n'est pas le cas et que la pièce $j$ est prête après la date $d_j$ alors il faudra payer une pénalité de $\omega_j$ par unité de temps de retard. On cherche à trouver l'ordre d'usinage des pièces minimisant la somme totale des pénalités de retard. \
Notons dés maintenant que ce problème peut être modélisé sous la forme d'un problème linéaire en nombres entiers que nous avons explicité ici :


**Variables:**

 * $\forall j \in J, r_j \ge 0$ est une variable réelle positive représentant le retard en unité de temps de la pièce $j$
 
 * $\forall j \in J, f_j \ge 0$ est une variable réelle positive représentant la date de fin d'usinage de la pièce $j$
 
 * $\forall (i,j)\in J^2$, $x_{i,j} \in \{0,1\}$ est une variable binaire prenant la valeur $1$ si la pièce $i$ passe avant la pièce $j$ et $0$ sinon

**Objectif:**

* On cherche à minimiser les pénalités de retard :

$$ \min{\sum_{j\in J}{\omega_j r_j}}$$

**Contraintes:**

* On impose la contrainte suivante pour calculer le retard de chaque pièce :
$$ r_j \ge f_j - d_j,\quad \forall j \in J$$

* On impose la contrainte suivatne pour calculer la date de fin d'usinage de chaque pièce :
$$ f_j = T_j + \sum_{i \in J}{T_i x_{ij}} \quad \forall j \in J$$

* On impose enfin une relation d'ordre total sur le passage des pièces :
$$ x_{ij} + x_{ji} = 1 \quad \forall (i,j) \in J^2, \: i < j$$
$$ x_{ik} \ge x_{ij} + x_{jk}-1\quad \forall (i,j,k) \in J^3, \: i<j, \: i<k, j\neq k$$

\
Nos implémentations ont été réalisé sous Python et nous avons fait appel (lorsque c'était nécessaire) au package PuLP pour résoudres des modèles de PLNE.

Nous avons dans un premier temps créée une structure de données permettant de représenter une instance du problème d'ordonnacement présenté ci-dessus. Nous avons ensuite créée une structure de données générique pour l'arbre d'énumération qui sera utilisé dans l'algorithme de Branch-and-Bound. Avec celle-ci, nous avons par la suite, implémenté l'algorithme de Branch-and-bound de manière générique en fragmentant ses différentes spécificités dans des sous-fonctions, ceci nous a notament permis de tester plusieurs méthodes pour chacune d'entre elles et ainsi comparer leurs performances sur des instances du problème d'ordonnancement. Les résultats et analyses de nos tests sont détaillés dans ce notebook. Enfin nous avons programmé des fonctions permettant de visualiser le parcours de l'arbre de branchement de l'algorithme de Branch-And-Bound selon les méthodes utilisées. Nous présenterons également une résolution de l'extension du problème d'ordonnancement dans laquelle on ajoute des contraintes de précédence entre certaines paires de pièces à usiner.

## I - Structure de données générique pour le problème d'ordonnancement

    TO DO

## II - Structure de données générique pour l'arbre d'énumération de l'algorithme de Branch-And-Bound

Après avoir implémenté la structure de données génériques pour le problème d'ordonnancement, nous nous sommes intéressé à la mise en place de l'algorithme de Branch-and-Bound. Premièrement, nous devions créer une structure de données génériques pour l'arbre d'énumération.

Pour cela, nous avons séparé la création de l'arbre en 2 étapes. Tout d'abord, nous avons crée une classe Noeud. Cette classe noeud regroupe les informations dont nous avons besoin pour chaque noeud :

* **description** : titre du noeud.


* **position** : elle permet de savoir si le noeud est à l'extrémité de l'arbre ( feuille) ou non ( branche ).


* **indice_pere** : elle nous donne la position du noeud père dans la liste des noeuds que nous allons vous présenter par la suite dans une deuxième classe. Le noeud racine prendra la valeur -1.


* **info** : cette variable permet de décrire un noeud. Dans notre cas, elle contiendra, la plupart du temps, l'ordre des pièces déjà fixées.

Voici l'implémentation de la classe Noeud en python :

In [2]:
class Noeud:
    def __init__(self,description,pos,info =[],indice=-1):
        self.description=description
        self.position = pos #branche : 0 ou une feuille : 1
        self.indice_pere = indice #-1 si noeud racine
        self.info = info

Ensuite, dans un second temps, nous avons crée la classe ArbreEnumeration. Cette classe permet de définir l'arbre qui va être utilisé par la suite dans l'algorithme de Branch-and-Bound. Celle-ci va donc permettre de stocker tous les noeuds de l'arbre. 
Chaque instance de la classe est constituée des attributs suivants :

* **nombre_noeuds** : nombre de noeuds de l'arbre crée suite à l'algorithme B&B


* **nombre_noeuds_non_traites** : nombre de noeuds qui n'ont pas encore été traités


* **noeuds** : liste regroupant tous les noeuds


* **indice_noeuds_non_traites** : liste regroupant les indices de la liste noeud des noeuds qui n'ont pas encore été traités


* **dual_bound** : // a completer


* **primal_bound** : borne primale du problème


Voici l'implémentation de la classe ArbreEnumeration en python : 

In [ ]:
class ArbreEnumeration:
    
    def __init__(self,d1,p1):
            self.nombre_noeuds=1
            self.nombre_noeuds_non_traites = 1
            self.noeuds = [Noeud("Problème initiale",False)]
            self.indice_noeuds_non_traites= [0]
            self.dual_bound=d1
            self.primal_bound=p1

## III - Implémentation de l'algorithme de Branch-And-Bound dans le cadre du problème d'ordonnancement

### a) Méthode 1 *(à détailler )* 
    TO DO
### b) Méthode 2 *(à détailler )*
    TO DO
### etc...

## IV - Visualisation du parcours de l'arbre de branchement lors de l'exécution de l'algorithme de Branch-And-Bound

    TO DO

## V - Ajout de contraintes de précédence sur les pièces au problème d'ordonnancement 

    TO DO

## Conclusion 

    TO DO